In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24326
2,Huelva,Confirmados PDIA 14 días,894
3,Huelva,Tasa PDIA 14 días,"174,21127501607654"
4,Huelva,Confirmados PDIA 7 días,279
5,Huelva,Total Confirmados,24554
6,Huelva,Curados,14436
7,Huelva,Fallecidos,278


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24326.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6498.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1893 personas en los últimos 7 días 

Un positivo PCR cada 498 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24326.0,279.0,894.0,513170.0,54.367948,174.211275,65.0
Huelva-Costa,14835.0,162.0,582.0,289548.0,55.949273,201.002943,36.0
Condado-Campiña,6762.0,105.0,257.0,156231.0,67.208173,164.500003,27.0
Huelva (capital),6498.0,76.0,289.0,143837.0,52.837587,200.921877,17.0
Almonte,835.0,28.0,71.0,24507.0,114.253071,289.713143,9.0
Moguer,809.0,18.0,41.0,21867.0,82.315818,187.497142,6.0
Ayamonte,1183.0,13.0,38.0,21104.0,61.599697,180.060652,5.0
Bollullos Par del Condado,594.0,18.0,41.0,14387.0,125.112949,284.979495,3.0
Lepe,1967.0,22.0,84.0,27880.0,78.909613,301.291248,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Escacena del Campo,138.0,5.0,9.0,2287.0,218.627022,393.528640,2.0
Villablanca,169.0,6.0,11.0,2885.0,207.972270,381.282496,2.0
Villanueva de los Castillejos,147.0,3.0,10.0,2825.0,106.194690,353.982301,2.0
Lepe,1967.0,22.0,84.0,27880.0,78.909613,301.291248,3.0
Cartaya,917.0,22.0,59.0,20083.0,109.545387,293.780810,2.0
Almonte,835.0,28.0,71.0,24507.0,114.253071,289.713143,9.0
Bollullos Par del Condado,594.0,18.0,41.0,14387.0,125.112949,284.979495,3.0
Rociana del Condado,305.0,7.0,19.0,7939.0,88.172314,239.324852,2.0
Isla Cristina,1973.0,5.0,48.0,21393.0,23.372131,224.372458,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Valverde del Camino,414.0,1.0,10.0,12750.0,7.843137,78.431373,0.0,0.100000
Isla Cristina,1973.0,5.0,48.0,21393.0,23.372131,224.372458,2.0,0.104167
Sierra de Huelva-Andévalo Central,2300.0,6.0,34.0,67391.0,8.903266,50.451841,1.0,0.176471
Aracena,453.0,1.0,5.0,8255.0,12.113870,60.569352,0.0,0.200000
Aljaraque,915.0,4.0,17.0,21474.0,18.627177,79.165502,0.0,0.235294
San Juan del Puerto,355.0,1.0,4.0,9411.0,10.625863,42.503453,0.0,0.250000
Gibraleón,508.0,4.0,16.0,12737.0,31.404569,125.618277,0.0,0.250000
Lepe,1967.0,22.0,84.0,27880.0,78.909613,301.291248,3.0,0.261905
Huelva (capital),6498.0,76.0,289.0,143837.0,52.837587,200.921877,17.0,0.262976
